In [1]:
#363 vs 361
# with and without transition statement
# with and without response times
# with and without I don't know only.
# with random names and with sequential user1, user2, etc...
# should include username to demographics

In [2]:
import pandas as pd

df = pd.read_csv("experiment_data/sqb/sqb_data.csv")

In [3]:
sqb_markers = ["National Highway Safety", "You are a serious reader", "When evaluating teaching job offers,"]

In [4]:
class Participant:

    transcript = ""
    transcript_with_response_times = ""
    choice_problem = ""

    def process_transcript(self, chat_transcript):
        # Transcript without response times
        transcript = ""
        for msg in chat_transcript:
            if any(marker in msg['content'] for marker in sqb_markers):
                break
            transcript += f"{msg['role']}: {msg['content']}\n"

        # Transcript with response times
        transcript_with_response_times = ""
        for i, msg in enumerate(chat_transcript):

            if any(marker in msg['content'] for marker in sqb_markers):
                break

            time_diff = None

            if i > 1:
                if msg['timestamp']['client_timestamp'] != 0 and chat_transcript[i-1]['timestamp']['client_timestamp'] != 0:
                    time_diff = msg['timestamp']['client_timestamp'] - chat_transcript[i-1]['timestamp']['client_timestamp']
                else:
                    try:
                        time_diff = msg['timestamp']['server_timestamp'] - chat_transcript[i-1]['timestamp']['server_timestamp']
                    except KeyError:
                        time_diff = -1

            # Add a blank line before each message except the first
            if i > 0:
                transcript_with_response_times += "\n"

            if msg['role'] == 'user' and i > 1:
                transcript_with_response_times += f"{msg['role']}: {msg['content']} (response time: {time_diff} ms)"
            else:
                transcript_with_response_times += f"{msg['role']}: {msg['content']}"

        # Extract the choice problem statement
        choice_problem = "\n".join(f"{msg['role']}: {msg['content']}" for msg in chat_transcript if any(marker in msg['content'] for marker in sqb_markers))
        choice_problem = choice_problem.replace("assistant: ", "")

        self.transcript = transcript
        self.transcript_with_response_times = transcript_with_response_times
        self.choice_problem = choice_problem


    def __init__(self, participant_id = None, condition = None, prolific_data = None, chat_transcript = None, change_username = None, change_assistantname = None):
        # Unique identifier for the participant
        self.Participant_ID = participant_id

        # Experiment condition
        self.condition = condition

        # Demographic info from Prolific
        self.prolific_data = prolific_data

        # Chat Data
        self.chat_transcript = chat_transcript if chat_transcript is not None else []

        if self.chat_transcript is not None:
            self.process_transcript(self.chat_transcript)

        if change_username is not None:
            self.change_user_name(change_username)

        if change_assistantname is not None:
            self.change_assistant_name(change_assistantname)

    def update_participant_id(self, participant_id):
        self.Participant_ID = participant_id

    def update_condition(self, condition):
        self.condition = condition

    def update_prolific_data(self, key, value):
        if self.prolific_data is not None:
            self.prolific_data[key] = value

    def update_chat_transcript(self, chat_transcript):
        self.chat_transcript = self.process_transcript(chat_transcript)

    def get_participant_id(self):
        return self.Participant_ID

    def get_demographics(self):
        return self.prolific_data

    def get_transcript(self) -> str:
        return self.transcript
    
    def get_transcript_with_response_times(self) -> str:
        return self.transcript_with_response_times

    def get_choice_problem(self) -> str:
        return self.choice_problem
    
    def change_user_name(self, new_name):
        self.transcript = self.transcript.replace("user:", f"{new_name}:")
        self.transcript_with_response_times = self.transcript_with_response_times.replace("user:", f"{new_name}:")
    
    def change_assistant_name(self, new_name):
        self.transcript = self.transcript.replace("assistant:", f"{new_name}:")
        self.transcript_with_response_times = self.transcript_with_response_times.replace("assistant:", f"{new_name}:")

columns = [
    "Age",
    "Sex",
    "Ethnicity simplified",
    "Country of birth",
    "Country of residence",
    "Nationality",
    "Language",
    "Student status",
    "Employment status"
]

participants = []

import ast

for row_id in range(len(df)):

    participant = Participant(participant_id = df["Participant id"][row_id], 
                            condition = df["expcode"][row_id], 
                            prolific_data = {col: df.loc[row_id, col] for col in columns}, 
                            chat_transcript = ast.literal_eval(df["chat_data"][row_id]),
                            change_username = "[User]",
                            change_assistantname = "[Interviewer]"
                           )
    participants.append(participant)


In [5]:
# print(participants[0].get_demographics())
# print(participants[0].get_transcript())
# print(participants[0].get_transcript_with_response_times())
# print(participants[0].get_choice_problem())

In [8]:
from genagents.genagents import GenerativeAgent
import json
from tqdm import tqdm

sim_output = []

for participant in tqdm(participants[:20]):
    # Initialize a new agent
    agent = GenerativeAgent()
    # Update the agent's scratchpad with personal information
    agent.update_scratch({k: (v.item() if hasattr(v, "item") else v) for k, v in participant.get_demographics().items()})
    dialogue = [
        ("Interviewer", participant.get_choice_problem()),
    ]

    response = agent.utterance(dialogue, context=participant.get_transcript())

    sim_output.append({ participant.get_participant_id() : response })

    agent.save("experiment_agents/{}".format(participant.get_participant_id()))

    # break


100%|██████████| 20/20 [00:31<00:00,  1.55s/it]


In [9]:
sim_output

[{'3a6fd310cc79b06916fc1004a781c2f2d3cd9d2e8b8880e1': 'I think I would choose to invest in moderate-risk Company A. The potential for a 30% increase seems more appealing to me than the high-risk option, especially since it has a good chance of being unchanged as well.'},
 {'3a6a83449c73b76844f24604f28591abde9b967c8e88d4e6': "I think I would prefer to invest in moderate-risk Company A. It seems like a safer choice with a decent chance of growth, and I'm not comfortable with high-risk investments right now."},
 {'3a6fd3159c7aef3c11a41557f5dfc4a48999cc2b80d285e4': "I would choose to invest in moderate-risk Company A. Even though the potential gain isn't as high as Company B, the risk seems more manageable for my current situation."},
 {'3961d345987fb33d16fc1553a7d3c2a3db9fcc2d8189d7e3': 'I would choose to invest in moderate-risk Company A. It seems like a safer option with a decent chance of increasing in value.'},
 {'3a6fd246c12ce43f17f14551f282c0a3dfcb997b888ad0e1': "I would choose to i

In [ ]:
merged_output = {}
for d in sim_output:
    merged_output.update(d)
merged_output

merged_df = pd.DataFrame(list(merged_output.items()), columns=["Participant_ID", "Response"])
merged_df.head()

In [ ]:
merged_df.to_csv("experiment_data/sqb/sqb_simulated_responses_v3.csv", index=False)

#### TEST

In [ ]:
# # pid = 112
# # pid = 363
# pid = 1

# from genagents.genagents import GenerativeAgent
# import json


# agent = GenerativeAgent()

# agent.update_scratch({k: (v.item() if hasattr(v, "item") else v) for k, v in participants[pid].get_demographics().items()})

# dialogue = [
#     ("Interviewer", participants[pid].get_choice_problem()),
# ]

# response = agent.utterance(dialogue, context=participants[pid].get_transcript())

# print(response)

# agent.save("experiment_agents/{}".format(participants[pid].get_participant_id()))